In [22]:
def preprocessing_1(img):

    background=img.copy()
    rotated_list=[]
    #preprocessing cycle -1 :
    gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)#converting to gray scale
    _, thresh = cv2.threshold(gray,200, 255, cv2.THRESH_BINARY) #thresholding the image
    edges = cv2.Canny(thresh, 100, 200)# edge detection

    kernel = np.ones((5, 5), np.uint8) #matrix size for dilation and eroding 5 x 5

    dilated = cv2.dilate(edges, kernel, iterations=2) #dilation
    eroded = cv2.erode(dilated, kernel, iterations=1) #erosion
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # finds the contour
    cv2.drawContours(background, contours, -1, (0, 255, 0), 3) # draw a contour around the uno card
    #show_image(background,"cropped",cmap="grey")
    
    for contour in contours:
        rect = cv2.minAreaRect(contour)

        box = cv2.boxPoints(rect) # creating the box points
        box = box.astype(int) # getting the x, y value for each cornor points
        
        cv2.drawContours(background, [box], -1, (255, 0, 0), 3) # drawing a box

        #calculating the center point
        center_x = int(np.mean(box[:, 0]))
        center_y = int(np.mean(box[:, 1]))
    
        # Draw the center point
        cv2.circle(background, (center_x, center_y), 5, (0, 0, 255), -1)
    
        pt1, pt2, pt3, pt4 = box #four points of the rectangle
    
        # midpoint of pt1 - pt2
        mid1 = ((pt1[0] + pt2[0]) // 2, (pt1[1] + pt2[1]) // 2)
        
        # midpoint of pt3 - pt4
        mid2 = ((pt3[0] + pt4[0]) // 2, (pt3[1] + pt4[1]) // 2)
        
        #midpoint of pt1 - pt4
        mid3=((pt1[0] + pt4[0]) // 2, (pt1[1] + pt4[1]) // 2)
        
        #midpoint of pt2 - pt3
        mid4 = ((pt3[0] + pt2[0]) // 2, (pt3[1] + pt2[1]) // 2)
    
        # Draw the line between midpoints
        cv2.line(background, mid1, mid2, (255, 255, 0), 2)  
        cv2.line(background, mid3, mid4, (0, 255, 0), 2)
        
        #distance between points or length of the line
        line1_length=math.sqrt((mid2[0] - mid1[0])**2+(mid2[1] - mid1[1])**2)
        line2_length=math.sqrt((mid4[0] - mid3[0])**2+(mid4[1] - mid3[1])**2)
    
        #finding the longest line and setting those mid point to take angle
        if line1_length>line2_length:
            midpoint1=mid1
            midpoint2=mid2
        else:
            midpoint1=mid3
            midpoint2=mid4
            
        # length of reference line
        line_length = 100
        
        # Starting point is center
        start_point = (center_x, center_y)
        
        # End point straight up (90 degrees)
        end_point = (center_x, center_y - line_length)
        
        # Drawing the vertical reference line
        cv2.line(background, start_point, end_point, (0, 255, 255), 2)  # Yellow line
                        
        # Vector of midpoint line
        vector_x = midpoint2[0] - midpoint1[0]
        vector_y = midpoint2[1] - midpoint1[1]
        
        # Angle between vector and vertical line (0, -1)
        angle_rad = np.arctan2(vector_y, vector_x) - np.arctan2(-1, 0)
        angle_deg = np.degrees(angle_rad)
        angle_deg = angle_deg % 360 
                        
        # Getting image center
        (h, w) = background.shape[:2]
        image_center = (w // 2, h // 2)
        
        # rotation matrix
        M = cv2.getRotationMatrix2D(image_center, angle_deg, 1.0)
        
        # Perform the rotation
        rotated = cv2.warpAffine(background_copy, M, (w, h))
        #show_image(background,"cropped",cmap="grey")
        #show_image(rotated, "Rotated image")
        rotated_list.append(rotated)
    return rotated_list
            

In [23]:
def preprocessing_2(img):

    rotated=img.copy()
    final_img=[]
    # preprocessing - cycle2

    gray_2 = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY) 
    _, thresh_2 = cv2.threshold(gray_2,220, 255, cv2.THRESH_BINARY)             
    edges_2 = cv2.Canny(thresh_2, 100, 200)
                  
    kernel = np.ones((5, 5), np.uint8)
    
    dilated_2 = cv2.dilate(edges_2, kernel, iterations=2)
    eroded_2 = cv2.erode(dilated_2, kernel, iterations=1)
    
    contours, _ = cv2.findContours(eroded_2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Process only if we have at least one valid contour
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(rotated, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
        
        if area > 500:  # Filter small areas
                                   
            cropped = rotated[y:y+h, x:x+w]                       
            resize_img = cv2.resize(cropped, (224, 224))
            
            # preprocessing - cycle3
            
            gray_3 = cv2.cvtColor(resize_img, cv2.COLOR_BGR2GRAY)                       
            _, thresh_3 = cv2.threshold(gray_3,220, 255, cv2.THRESH_BINARY)
            edges_3 = cv2.Canny(thresh_3, 100, 200)
            
            kernel = np.ones((5, 5), np.uint8)
            
            dilated_3 = cv2.dilate(edges_3, kernel, iterations=2)
            eroded_3 = cv2.erode(dilated_3, kernel, iterations=1)
            
            contours, _ = cv2.findContours(thresh_3, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                area2 = cv2.contourArea(cnt)
                
                
                if area2> 19000 and area2 <30000:
                                        
                    background = np.ones((224, 224, 3), dtype=np.uint8) * 255  
                    
                    # Creating mask for the oval contour
                    mask = np.zeros_like(resize_img)
                    cv2.drawContours(mask, [cnt], 0, (255, 255, 255), -1)
                                                   
                    # Applying mask to extract oval region
                    masked = cv2.bitwise_and(resize_img, mask)
                    
                    # Crop the contour region
                    x, y, w, h = cv2.boundingRect(cnt)
                    cropped = masked[y:y+h, x:x+w]
                    
                    # Get shape of cropped region
                    ch, cw = cropped.shape[:2]
    
                    target_size = 224
                    # Compute offsets to center it
                    x_offset = (target_size - cw) // 2
                    y_offset = (target_size - ch) // 2
                    
                    # Pasting cropped oval into the center of white background
                    background[y_offset:y_offset+ch, x_offset:x_offset+cw] = cropped

                    
                    #final_crop=cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
                    gray_final = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
                    _, thresh_final = cv2.threshold(gray_final,226, 255, cv2.THRESH_BINARY)
                    #show_image(thresh_final,"final - image",cmap="grey")

                    # Resize to final model input size
                    #final_img = cv2.resize(thresh_final, (img_size, img_size))
                    final_img.append(thresh_final)
    return final_img

In [28]:
import os
import cv2 , math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load trained model
model = tf.keras.models.load_model("uno_number_detection.keras")

# Define categories
categories = ["Number_1", "Number_2", "Number_3","Number_4","Number_5","Number_6","Number_7","Number_8","Number_9","Symbol_Draw_2","Symbol_Reverse","Symbol_Skip"]

# Open Webcam
cap = cv2.VideoCapture(0)

# Set frame width and height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    edges = cv2.Canny(thresh, 100, 200)

    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=2)
    eroded = cv2.erode(dilated, kernel, iterations=1)

    contours, _ = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 2500:
            x, y, w, h = cv2.boundingRect(cnt)
            
            # Draw contour rectangle
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

            # Crop from original image
            cropped = frame[y:y+h, x:x+w]

            output_h, output_w = 640, 480
            background = np.zeros((output_h, output_w, 3), dtype=np.uint8)

            ch, cw = cropped.shape[:2]
            if ch <= output_h and cw <= output_w:
                y_offset = (output_h - ch) // 2
                x_offset = (output_w - cw) // 2
                background[y_offset:y_offset+ch, x_offset:x_offset+cw] = cropped
            #show_image(background,"cropped",cmap="grey")
                background_copy=background.copy()

                rotated_img=preprocessing_1(background)
                for allrotated in rotated_img:
                    
                    predict_img=preprocessing_2(allrotated)

                    for allimages in predict_img:
                        #show_image(allimages, "final image for prediction")
                        final_img = cv2.resize(allimages, (224, 224))
                        # Preprocess the image
                        img_processed = preprocess_input(final_img.astype(np.float32))
                        img_array = np.expand_dims(img_processed, axis=0)
    
                        predictions = model.predict(img_array, verbose=0)
                        class_id = np.argmax(predictions)
                        confidence = np.max(predictions)
                    
                        # Draw rectangle and label
                        label = f"{categories[class_id]}"#" ({confidence * 100:.2f}%)" 
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
    # Show video feed
    cv2.imshow("Live Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()   

In [29]:
import matplotlib.pyplot as plt

def show_image(img, title="Image", cmap=None):
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis("off")
    plt.show()